<a href="https://www.kaggle.com/code/vincemarcs/mvsa-image-models?scriptVersionId=101001812" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
SEED = 61


import os
import re
import gc
import h5py
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
import random as python_random
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

from tqdm import tqdm
from nltk import tokenize

from sklearn import preprocessing
from sklearn.decomposition import PCA
from IPython.display import display_html
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

from keras import backend as K
from keras import initializers,regularizers,constraints
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Reshape, Input, Embedding, Flatten, Dense, Dropout, BatchNormalization, Activation #, merge
from keras.layers import TimeDistributed, LSTM, GRU, Bidirectional, Convolution1D, MaxPooling1D, MaxPool2D, Convolution2D
from keras.layers.core import RepeatVector, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, Model, load_model
from sklearn.model_selection import cross_val_score

def reset_seeds():
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

# from tensorflow.python.keras.layers import Layer, InputSpec, Lambda
# from tensorflow.keras import Model
# from attention import Attention_input1, Attention_input2
# from keras.optimizers import SGD, RMSprop, Adagrad

In [2]:
def read_hdf5(path):
    read_file = h5py.File(path, 'r')

    feature_names = list(read_file.keys())
    loaded_data = []
    
    for name in feature_names:
        dataset = read_file[name][:]
        if dataset.dtype == np.dtype('object'):
            dataset = np.array([x.decode('UTF-8') for x in dataset])            
        loaded_data.append((name, dataset))

    return loaded_data

def loadz(path):
    data = np.load(path)['arr_0']
    return data

In [3]:
def load_labels(path):
    data = read_hdf5(path)

    for x in data:
        if x[0] == 'multimodal-labels':
            labels = x[1]
        if x[0] == 'text-labels':
            text_labels = x[1]
        if x[0] == 'image-labels':
            image_labels = x[1]

    return labels, text_labels, image_labels

def load_mvsa_feature(feature_name, merge=False):
    folder_path = os.path.join('../input/mvsa-features/', feature_name)
    single_file = 'mvsa-single-{}.npz'.format(feature_name)
    multiple_file = 'mvsa-multiple-{}.npz'.format(feature_name)
    mvsa_single = loadz(os.path.join(folder_path, single_file))
    mvsa_multiple = loadz(os.path.join(folder_path, multiple_file))
    
    if merge == True:
        return merge_mvsa(mvsa_single, mvsa_multiple)
    
    return mvsa_single, mvsa_multiple

def load_mvsa_images(merge=False):
    folder_path = '../input/mvsa-data'
    file_paths = os.listdir(folder_path)
    for path in file_paths:
        file_name = os.path.split(path)[1]
        if file_name.split('.')[1] == 'npz':
            if file_name.split('-')[1] == 'single':
                mvsa_single_images_path = os.path.join(folder_path, path)
            else:
                mvsa_multiple_images_path = os.path.join(folder_path, path)
    
    mvsa_single = loadz(mvsa_single_images_path)
    mvsa_multiple = loadz(mvsa_multiple_images_path)
    
    if merge == True:
        return merge_mvsa(mvsa_single, mvsa_multiple)
    
    return mvsa_single, mvsa_multiple

def merge_mvsa(mvsa_single, mvsa_multiple):
    mvsa = np.concatenate((mvsa_single, mvsa_multiple), axis=0)
    return mvsa

In [4]:
def plot_metrics(history):
    fig = plt.figure(figsize=(20, 5))

    fig.add_subplot(1, 4, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('LOSS')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('ACCURACY')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 3)
    plt.plot(history.history['f1_macro'])
    plt.plot(history.history['val_f1_macro'])
    plt.title('Macro F1-SCORE')
    plt.ylabel('f1-macro')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')
    
    fig.add_subplot(1, 4, 4)
    plt.plot(history.history['f1_weighted'])
    plt.plot(history.history['val_f1_weighted'])
    plt.title('Weighted F1-SCORE')
    plt.ylabel('f1-weighted')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    plt.show()

In [5]:
# e.g. validation_split=0.1 -----> 8:1:1 ratio of train, val, test
def split_data(data, validation_split):
    num_val = int(validation_split * len(data))
    data_train = data[:-(num_val*2)]
    data_val = data[-(num_val*2):-(num_val)]
    data_test = data[-num_val:]
    return data_train, data_val, data_test

# e.g. validation_split=0.1 -----> 8:1:1 ratio of train, val, test
def split_tf_data(data, validation_split):
    DATASET_SIZE = len(data)
    train_size = int((1-validation_split*2) * DATASET_SIZE)
    val_size = int(validation_split * DATASET_SIZE)
    test_size = int(validation_split * DATASET_SIZE)

#     full_dataset = tf.data.TFRecordDataset(FLAGS.input_file)
#     full_dataset = full_dataset.shuffle()
    train_dataset = data.take(train_size)
    test_dataset = data.skip(train_size)
    val_dataset = test_dataset.skip(test_size)
    test_dataset = test_dataset.take(test_size)
    return train_dataset, val_dataset, test_dataset

In [6]:
def create_model_pretrained(input_shape):
    f1_macro = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro', name='f1_macro')
    f1_weighted = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='weighted', name='f1_weighted')

    image_input = Input(shape=input_shape)
    outputs = Dense(NUM_CLASSES, activation='softmax') (image_input)
    
    model = Model(image_input, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_macro, f1_weighted])
    return model

def create_model_cnn(input_shape):
    f1_macro = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro', name='f1_macro')
    f1_weighted = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='weighted', name='f1_weighted')

    image_inputs = Input(shape=input_shape)
#     x = Reshape((1, input_shape[0], input_shape[1], input_shape[2])) (image_inputs)
    x = Convolution2D(32, kernel_size=(5,5), strides=(1,1), activation='relu') (image_inputs)
#     x = Reshape((x.shape[2], x.shape[3], x.shape[4])) (x)
    x = MaxPool2D(pool_size=(2,2)) (x)
    x = Convolution2D(64, kernel_size=(5,5), strides=(1,1), activation='relu') (x)
    x = MaxPool2D(pool_size=(2,2)) (x)
    x = Convolution2D(128, kernel_size=(5,5), strides=(1,1), activation='relu') (x)
    x = MaxPool2D(pool_size=(2,2)) (x)
    x = Flatten() (x)
    x = Dense(64, activation='relu') (x)
    outputs = Dense(NUM_CLASSES, activation='softmax') (x)

    model = Model(image_inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_macro, f1_weighted])
    return model

In [7]:
def evaluate_model(model, X_test, y_test, checkpoint=None, verbose=1):
    if checkpoint is not None:
        model = load_model('./model_checkpoint/{}.h5'.format(checkpoint))
    
    loss, acc, f1_macro, f1_weighted = model.evaluate(X_test, y_test, verbose=verbose)
    
    if verbose == 1:
        print('Loss:', loss)
        print('Accuracy:', acc)
        print('Macro F1-score:', f1_macro)
        print('Weighted F1-score:', f1_weighted)
        
    return loss, acc, f1_macro, f1_weighted

In [8]:
def run_and_evaluate(name, X, y, verbose=0):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))
    
    random_idx = np.random.permutation(len(y))
    X, y = X[random_idx], y[random_idx]

    X_train, X_val, X_test = split_data(X, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)
    
    oversample = BorderlineSMOTE(sampling_strategy='all', random_state=SEED, kind='borderline-1')
#     oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
    X_train, y_train = oversample.fit_resample(X_train, y_train)
    
    model = create_model_pretrained(X_train.shape[1:])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    checkpoint = ModelCheckpoint('./model_checkpoint/{}.h5'.format(name), save_best_only=True, verbose=verbose)
    
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                                   epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=verbose,
                                   callbacks=[checkpoint, early_stopping])
    if verbose == 1:
        best_epoch = np.argmin(history.history['val_loss'])
        print('Checkpoint loaded at epoch:', best_epoch)
    
    return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

In [9]:
def run_and_evaluate_cnn(name, X, y, verbose=0):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))
    
    random_idx = np.random.permutation(len(y))
    X, y = X[random_idx], y[random_idx]

    X_train, X_val, X_test = split_data(X, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)
    
    oversample = BorderlineSMOTE(sampling_strategy='minority', random_state=SEED, kind='borderline-1')
#     oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
    X_train, y_train = oversample.fit_resample(X_train, y_train)
    
    train_set = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
    val_set = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)
    test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)
    
    model = create_model_cnn(X_train.shape[1:])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    checkpoint = ModelCheckpoint('./model_checkpoint/{}.h5'.format(name), save_best_only=True, verbose=verbose)
    
    history = model.fit(train_set, validation_data=val_set, 
                                   epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=verbose,
                                   callbacks=[checkpoint, early_stopping])
    if verbose == 1:
        best_epoch = np.argmin(history.history['val_loss'])
        print('Checkpoint loaded at epoch:', best_epoch)
    
#     return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

In [10]:
def style_dataframe(dataframe):
    return dataframe.style.highlight_max(subset=['Accuracy', 'F1-macro', 'F1-weighted'], props='color:lawngreen', axis=0)\
                          .highlight_min(subset=['Accuracy', 'F1-macro', 'F1-weighted'], props='color:tomato', axis=0)\
                          .highlight_min(subset=['Loss'], props='color:lawngreen', axis=0)\
                          .highlight_max(subset=['Loss'], props='color:tomato', axis=0)

def display_dataframes(dfs, names=[], index=False):
    def to_df(x):
        if isinstance(x, pd.Series):
            return pd.DataFrame(x)
        else:
            return x
    html_str = ''
    if names:
        html_str += ('<tr>' + 
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
                     '</tr>')
    html_str += ('<tr>' + 
                 ''.join(f'<td style="vertical-align:top"> {to_df(df).to_html()}</td>' 
                         for df in dfs) + 
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

# Load data

In [11]:
mvsa_single_multimodal_labels, mvsa_single_text_labels, mvsa_single_image_labels = load_labels('../input/mvsa-features/labels/mvsa-single-labels.hdf5')
mvsa_multiple_multimodal_labels, mvsa_multiple_text_labels, mvsa_multiple_image_labels = load_labels('../input/mvsa-features/labels/mvsa-multiple-labels.hdf5')

mvsa_multimodal_labels = merge_mvsa(mvsa_single_multimodal_labels, mvsa_multiple_multimodal_labels)
mvsa_text_labels = merge_mvsa(mvsa_single_text_labels, mvsa_multiple_text_labels)
mvsa_image_labels = merge_mvsa(mvsa_single_image_labels, mvsa_multiple_image_labels)

le = preprocessing.LabelEncoder()
le.fit(mvsa_multimodal_labels)
NUM_CLASSES = len(le.classes_) # =3

In [12]:
# prepare all features data
feature_names = ['xception', 'vgg16', 'vgg19', 'resnet50', 'resnet101', 'resnet152', 'densenet121', 'densenet169', 'densenet201']

mvsa_single_features = []
mvsa_multiple_features = []
mvsa_features = []

for name in tqdm(feature_names):
    data = load_mvsa_feature(name)
    merge_data = merge_mvsa(data[0], data[1])

    mvsa_single_features.append(data[0])
    mvsa_multiple_features.append(data[1])
    mvsa_features.append(merge_data)

100%|██████████| 9/9 [00:22<00:00,  2.48s/it]


# Run models and Evalution display

In [13]:
reset_seeds()
EPOCHS = 100
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.1
EARLY_STOPPING = 10

## With original image labels

In [14]:
print('MVSA-Single: With original image labels')
mvsa_single_histories = []
mvsa_single_scores = []
for i in range(len(feature_names)):        
    print('MVSA-Single:', feature_names[i])
    if feature_names[i] == 'cnn':
        history, score = run_and_evaluate_cnn('single-OL-' + feature_names[i], mvsa_single_features[i], mvsa_single_image_labels, verbose=0)
    else:
        history, score = run_and_evaluate('single-OL-' + feature_names[i], mvsa_single_features[i], mvsa_single_image_labels, verbose=0)
    mvsa_single_histories.append(history)
    mvsa_single_scores.append(score)
    print()
df_single_scores = pd.DataFrame(mvsa_single_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

print('MVSA-Multiple: With original image labels')
mvsa_multiple_histories = []
mvsa_multiple_scores = []
for i in range(len(feature_names)):
#     print('MVSA-Multiple:', feature_names[i])
    if feature_names[i] == 'cnn':
        history, score = run_and_evaluate_cnn('multiple-OL-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_image_labels, verbose=1)
    else:
        history, score = run_and_evaluate('multiple-OL-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_image_labels, verbose=0)
    mvsa_multiple_histories.append(history)
    mvsa_multiple_scores.append(score)
    print()
df_multiple_scores = pd.DataFrame(mvsa_multiple_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

mvsa_average_scores = np.mean([mvsa_single_scores, mvsa_multiple_scores], axis=0)
df_average_scores = pd.DataFrame(mvsa_average_scores, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

MVSA-Single: With original image labels
MVSA-Single: xception


2022-07-16 18:09:30.951965: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-16 18:09:30.953084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-16 18:09:30.953761: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-16 18:09:30.954622: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil


MVSA-Single: vgg16

MVSA-Single: vgg19

MVSA-Single: resnet50

MVSA-Single: resnet101

MVSA-Single: resnet152

MVSA-Single: densenet121

MVSA-Single: densenet169

MVSA-Single: densenet201

MVSA-Multiple: With original image labels



2022-07-16 18:11:12.421147: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 373161984 exceeds 10% of free system memory.
2022-07-16 18:11:12.853267: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 373161984 exceeds 10% of free system memory.


2022-07-16 18:11:47.149234: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 375963648 exceeds 10% of free system memory.
2022-07-16 18:11:47.532453: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 375963648 exceeds 10% of free system memory.


## With multimodal labels

In [15]:
print('MVSA-Single: With multimodal labels')
mvsa_single_histories2 = []
mvsa_single_scores2 = []
for i in range(len(feature_names)):
    print('MVSA-Single:', feature_names[i])
    history, score = run_and_evaluate('single-ML-' + feature_names[i], mvsa_single_features[i], mvsa_single_multimodal_labels, verbose=0)
    mvsa_single_histories2.append(history)
    mvsa_single_scores2.append(score)
    print()
df_single_scores2 = pd.DataFrame(mvsa_single_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

print('MVSA-Multiple: With multimodal labels')
mvsa_multiple_histories2 = []
mvsa_multiple_scores2 = []
for i in range(len(feature_names)):
    print('MVSA-Multiple:', feature_names[i])
    history, score = run_and_evaluate('multiple-ML-' + feature_names[i], mvsa_multiple_features[i], mvsa_multiple_multimodal_labels, verbose=0)
    mvsa_multiple_histories2.append(history)
    mvsa_multiple_scores2.append(score)
    print()
df_multiple_scores2 = pd.DataFrame(mvsa_multiple_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

mvsa_average_scores2 = np.mean([mvsa_single_scores2, mvsa_multiple_scores2], axis=0)
df_average_scores2 = pd.DataFrame(mvsa_average_scores2, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

MVSA-Single: With multimodal labels
MVSA-Single: xception

MVSA-Single: vgg16

MVSA-Single: vgg19

MVSA-Single: resnet50

MVSA-Single: resnet101

MVSA-Single: resnet152

MVSA-Single: densenet121

MVSA-Single: densenet169

MVSA-Single: densenet201

MVSA-Multiple: With multimodal labels
MVSA-Multiple: xception

MVSA-Multiple: vgg16


2022-07-16 18:16:00.356342: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 445956096 exceeds 10% of free system memory.



MVSA-Multiple: vgg19

MVSA-Multiple: resnet50

MVSA-Multiple: resnet101

MVSA-Multiple: resnet152

MVSA-Multiple: densenet121

MVSA-Multiple: densenet169

MVSA-Multiple: densenet201



## With merge MVSA data

In [16]:
print('Both MVSA: With original image labels')
mvsa_histories3 = []
mvsa_scores3 = []
for i in range(len(feature_names)):
    print('Both MVSA:', feature_names[i])
    history, score = run_and_evaluate('merge-OL-' + feature_names[i], mvsa_features[i], mvsa_image_labels, verbose=0)
    mvsa_histories3.append(history)
    mvsa_scores3.append(score)
    print()
df_scores3 = pd.DataFrame(mvsa_scores3, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

print('Both MVSA: With multimodal labels')
mvsa_histories4 = []
mvsa_scores4 = []
for i in range(len(feature_names)):
    print('Both MVSA:', feature_names[i])
    history, score = run_and_evaluate('merge-ML-' + feature_names[i], mvsa_features[i], mvsa_multimodal_labels, verbose=0)
    mvsa_histories4.append(history)
    mvsa_scores4.append(score)
    print()
df_scores4 = pd.DataFrame(mvsa_scores4, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

Both MVSA: With original image labels
Both MVSA: xception

Both MVSA: vgg16

Both MVSA: vgg19

Both MVSA: resnet50

Both MVSA: resnet101

Both MVSA: resnet152

Both MVSA: densenet121

Both MVSA: densenet169

Both MVSA: densenet201

Both MVSA: With multimodal labels
Both MVSA: xception

Both MVSA: vgg16

Both MVSA: vgg19

Both MVSA: resnet50

Both MVSA: resnet101

Both MVSA: resnet152

Both MVSA: densenet121

Both MVSA: densenet169

Both MVSA: densenet201



# Display results

In [17]:
print('With original image labels\n')
display_dataframes((style_dataframe(df_single_scores), style_dataframe(df_multiple_scores), style_dataframe(df_average_scores)), 
                   names=['MVSA-Single', 'MVSA-Multiple', 'Average'])

With original image labels



MVSA-Single MVSA-Multiple Average 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.885935 
 0.596452 
 0.555707 
 0.600417 
 
 
 vgg16 
 0.915530 
 0.589800 
 0.503804 
 0.574620 
 
 
 vgg19 
 0.938491 
 0.554324 
 0.510197 
 0.560972 
 
 
 resnet50 
 1.030845 
 0.492239 
 0.426054 
 0.477299 
 
 
 resnet101 
 1.009535 
 0.492239 
 0.416971 
 0.481815 
 
 
 resnet152 
 1.005284 
 0.538803 
 0.440674 
 0.509006 
 
 
 densenet121 
 0.917368 
 0.538803 
 0.498408 
 0.538018 
 
 
 densenet169 
 0.866064 
 0.598670 
 0.539423 
 0.601144 
 
 
 densenet201 
 0.908708 
 0.596452 
 0.542895 
 0.600226 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.897259 
 0.555817 
 0.472824 
 0.565649 
 
 
 vgg16 
 0.875248 
 0.535253 
 0.434659 
 0.547653 
 
 
 vgg19 
 0.964263 
 0.495300 
 0.402188 
 0.505380 
 
 
 resnet50 
 0.973841 
 0.508226 
 0.387464 
 0.508294 
 
 
 resnet101 
 1.048069 
 0.448884 
 0.337438 
 0.440973 
 
 
 resnet152 
 0.999393 
 0.484136 
 0.338297 
 0.466906 
 
 
 densenet121 
 0.940360 
 0.530552 
 0.459009 
 0.544987 
 
 
 densenet169 
 0.916665 
 0.538190 
 0.446783 
 0.547926 
 
 
 densenet201 
 0.905160 
 0.559342 
 0.453737 
 0.559233 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.891597 
 0.576135 
 0.514265 
 0.583033 
 
 
 vgg16 
 0.895389 
 0.562527 
 0.469232 
 0.561136 
 
 
 vgg19 
 0.951377 
 0.524812 
 0.456193 
 0.533176 
 
 
 resnet50 
 1.002343 
 0.500233 
 0.406759 
 0.492797 
 
 
 resnet101 
 1.028802 
 0.470562 
 0.377204 
 0.461394 
 
 
 resnet152 
 1.002338 
 0.511469 
 0.389486 
 0.487956 
 
 
 densenet121 
 0.928864 
 0.534677 
 0.478708 
 0.541503 
 
 
 densenet169 
 0.891365 
 0.568430 
 0.493103 
 0.574535 
 
 
 densenet201 
 0.906934 
 0.577897 
 0.498316 
 0.579729

In [18]:
print('With multimodal labels\n')
display_dataframes((style_dataframe(df_single_scores2), style_dataframe(df_multiple_scores2), style_dataframe(df_average_scores2)), 
                   names=['MVSA-Single', 'MVSA-Multiple', 'Average'])

With multimodal labels



MVSA-Single MVSA-Multiple Average 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.849619 
 0.605322 
 0.528580 
 0.597583 
 
 
 vgg16 
 0.931811 
 0.541020 
 0.453357 
 0.534640 
 
 
 vgg19 
 0.885785 
 0.583149 
 0.453791 
 0.571318 
 
 
 resnet50 
 1.000087 
 0.503326 
 0.416259 
 0.515286 
 
 
 resnet101 
 1.051844 
 0.441242 
 0.266394 
 0.402038 
 
 
 resnet152 
 0.988817 
 0.463415 
 0.346739 
 0.485961 
 
 
 densenet121 
 0.975514 
 0.545455 
 0.465380 
 0.551436 
 
 
 densenet169 
 0.931655 
 0.538803 
 0.468508 
 0.554392 
 
 
 densenet201 
 0.901887 
 0.592018 
 0.477472 
 0.598573 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.928790 
 0.557579 
 0.427969 
 0.571629 
 
 
 vgg16 
 0.860475 
 0.607521 
 0.422769 
 0.595890 
 
 
 vgg19 
 0.870453 
 0.597532 
 0.394028 
 0.570353 
 
 
 resnet50 
 1.000586 
 0.502938 
 0.376374 
 0.529782 
 
 
 resnet101 
 0.992200 
 0.564042 
 0.358589 
 0.550071 
 
 
 resnet152 
 0.961785 
 0.551704 
 0.361083 
 0.556991 
 
 
 densenet121 
 0.958468 
 0.538190 
 0.420368 
 0.567701 
 
 
 densenet169 
 0.992213 
 0.494125 
 0.406654 
 0.525974 
 
 
 densenet201 
 0.865515 
 0.585194 
 0.464563 
 0.601871 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.889205 
 0.581450 
 0.478274 
 0.584606 
 
 
 vgg16 
 0.896143 
 0.574270 
 0.438063 
 0.565265 
 
 
 vgg19 
 0.878119 
 0.590340 
 0.423909 
 0.570835 
 
 
 resnet50 
 1.000336 
 0.503132 
 0.396316 
 0.522534 
 
 
 resnet101 
 1.022022 
 0.502642 
 0.312492 
 0.476055 
 
 
 resnet152 
 0.975301 
 0.507559 
 0.353911 
 0.521476 
 
 
 densenet121 
 0.966991 
 0.541822 
 0.442874 
 0.559568 
 
 
 densenet169 
 0.961934 
 0.516464 
 0.437581 
 0.540183 
 
 
 densenet201 
 0.883701 
 0.588606 
 0.471017 
 0.600222

In [19]:
print('With both MVSA merged together\n')
display_dataframes((style_dataframe(df_scores3), style_dataframe(df_scores4)), 
                   names=['Original image labels', 'Multimodal labels'])

With both MVSA merged together



Original image labels Multimodal labels 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.936570 
 0.549930 
 0.488696 
 0.560190 
 
 
 vgg16 
 0.929966 
 0.555968 
 0.407684 
 0.514177 
 
 
 vgg19 
 0.935760 
 0.546679 
 0.435069 
 0.536652 
 
 
 resnet50 
 0.995600 
 0.522527 
 0.411532 
 0.523537 
 
 
 resnet101 
 1.004458 
 0.490478 
 0.390454 
 0.492986 
 
 
 resnet152 
 0.990556 
 0.549466 
 0.349796 
 0.486822 
 
 
 densenet121 
 0.910823 
 0.560149 
 0.488416 
 0.569414 
 
 
 densenet169 
 0.958507 
 0.520204 
 0.456153 
 0.532004 
 
 
 densenet201 
 0.929598 
 0.553182 
 0.494687 
 0.559505 
 
 
 
 
 
 
 
   
 Loss 
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 xception 
 0.952643 
 0.539248 
 0.448127 
 0.561141 
 
 
 vgg16 
 0.983792 
 0.513237 
 0.408007 
 0.522903 
 
 
 vgg19 
 0.923719 
 0.574083 
 0.431408 
 0.573795 
 
 
 resnet50 
 0.989655 
 0.548072 
 0.420113 
 0.555084 
 
 
 resnet101 
 1.019213 
 0.499303 
 0.381173 
 0.516651 
 
 
 resnet152 
 0.991548 
 0.559220 
 0.388755 
 0.542013 
 
 
 densenet121 
 0.919839 
 0.582908 
 0.450378 
 0.583329 
 
 
 densenet169 
 0.915302 
 0.580121 
 0.460081 
 0.582531 
 
 
 densenet201 
 0.927737 
 0.571760 
 0.454906 
 0.586040

# Dratfs

In [20]:
# # load separate
# mvsa_single_images, mvsa_multiple_images = load_mvsa_images()
# mvsa_single_xception, mvsa_multiple_xception = load_mvsa_feature('xception')
# mvsa_single_vgg16, mvsa_multiple_vgg16 = load_mvsa_feature('vgg16')
# mvsa_single_vgg19, mvsa_multiple_vgg19 = load_mvsa_feature('vgg19')
# mvsa_single_resnet50, mvsa_multiple_resnet50 = load_mvsa_feature('resnet50')
# mvsa_single_resnet101, mvsa_multiple_resnet101 = load_mvsa_feature('resnet101')
# mvsa_single_resnet152, mvsa_multiple_resnet152 = load_mvsa_feature('resnet152')
# mvsa_single_densenet121, mvsa_multiple_densenet121 = load_mvsa_feature('densenet121')
# mvsa_single_densenet169, mvsa_multiple_densenet169 = load_mvsa_feature('densenet169')
# mvsa_single_densenet201, mvsa_multiple_densenet201 = load_mvsa_feature('densenet201')

# # load merge
# mvsa_images = merge_mvsa(mvsa_single_images, mvsa_multiple_images)
# mvsa_xception = merge_mvsa(mvsa_single_xception, mvsa_multiple_xception)
# mvsa_vgg16 = merge_mvsa(mvsa_single_vgg16, mvsa_multiple_vgg16)
# mvsa_vgg19 = merge_mvsa(mvsa_single_vgg19, mvsa_multiple_vgg19)
# mvsa_resnet50 = merge_mvsa(mvsa_single_resnet50, mvsa_multiple_resnet50)
# mvsa_resnet101 = merge_mvsa(mvsa_single_resnet101, mvsa_multiple_resnet101)
# mvsa_resnet152 = merge_mvsa(mvsa_single_resnet152, mvsa_multiple_resnet152)
# mvsa_densenet121 = merge_mvsa(mvsa_single_densenet121, mvsa_multiple_densenet121)
# mvsa_densenet169 = merge_mvsa(mvsa_single_densenet169, mvsa_multiple_densenet169)
# mvsa_densenet201 = merge_mvsa(mvsa_single_densenet201, mvsa_multiple_densenet201)

# # prepare all features data
# feature_names = ['cnn', 'xception', 'vgg16', 'vgg19', 'resnet50', 'resnet101', 'resnet152', 'densenet121', 'densenet169', 'densenet201']

# mvsa_single_features = [mvsa_single_images,
#                         mvsa_single_xception,
#                         mvsa_single_vgg16, mvsa_single_vgg19, 
#                         mvsa_single_resnet50, mvsa_single_resnet101, mvsa_single_resnet152, 
#                         mvsa_single_densenet121, mvsa_single_densenet169, mvsa_single_densenet201]

# mvsa_multiple_features = [mvsa_multiple_images,
#                           mvsa_multiple_xception,
#                           mvsa_multiple_vgg16, mvsa_multiple_vgg19, 
#                           mvsa_multiple_resnet50, mvsa_multiple_resnet101, mvsa_multiple_resnet152, 
#                           mvsa_multiple_densenet121, mvsa_multiple_densenet169, mvsa_multiple_densenet201]

# mvsa_features = [mvsa_images,
#                  mvsa_xception,
#                  mvsa_vgg16, mvsa_vgg19, 
#                  mvsa_resnet50, mvsa_resnet101, mvsa_resnet152, 
#                  mvsa_densenet121, mvsa_densenet169, mvsa_densenet201]